In [1]:
from keras.optimizers import Adam
from keras.models import Sequential
from keras.layers.core import Dense, Dropout
import random
import numpy as np
import pandas as pd
from operator import add
from numba import vectorize



class DQNAgent(object):

    def __init__(self):
        self.reward = 0
        self.gamma = 0.9
        self.dataframe = pd.DataFrame()
        self.short_memory = np.array([])
        self.agent_target = 1
        self.agent_predict = 0
        self.learning_rate = 0.0005
        self.model = self.network()
        #self.model = self.network("weights.hdf5")
        self.epsilon = 0
        self.actual = []
        self.memory = []

    def get_state(self, game, player, food):

        state = [
            (player.x_change == 20 and player.y_change == 0 and ((list(map(add, player.position[-1], [20, 0])) in player.position) or
            player.position[-1][0] + 20 >= (game.game_width - 20))) or (player.x_change == -20 and player.y_change == 0 and ((list(map(add, player.position[-1], [-20, 0])) in player.position) or
            player.position[-1][0] - 20 < 20)) or (player.x_change == 0 and player.y_change == -20 and ((list(map(add, player.position[-1], [0, -20])) in player.position) or
            player.position[-1][-1] - 20 < 20)) or (player.x_change == 0 and player.y_change == 20 and ((list(map(add, player.position[-1], [0, 20])) in player.position) or
            player.position[-1][-1] + 20 >= (game.game_height-20))),  # danger straight

            (player.x_change == 0 and player.y_change == -20 and ((list(map(add,player.position[-1],[20, 0])) in player.position) or
            player.position[ -1][0] + 20 > (game.game_width-20))) or (player.x_change == 0 and player.y_change == 20 and ((list(map(add,player.position[-1],
            [-20,0])) in player.position) or player.position[-1][0] - 20 < 20)) or (player.x_change == -20 and player.y_change == 0 and ((list(map(
            add,player.position[-1],[0,-20])) in player.position) or player.position[-1][-1] - 20 < 20)) or (player.x_change == 20 and player.y_change == 0 and (
            (list(map(add,player.position[-1],[0,20])) in player.position) or player.position[-1][
             -1] + 20 >= (game.game_height-20))),  # danger right

             (player.x_change == 0 and player.y_change == 20 and ((list(map(add,player.position[-1],[20,0])) in player.position) or
             player.position[-1][0] + 20 > (game.game_width-20))) or (player.x_change == 0 and player.y_change == -20 and ((list(map(
             add, player.position[-1],[-20,0])) in player.position) or player.position[-1][0] - 20 < 20)) or (player.x_change == 20 and player.y_change == 0 and (
            (list(map(add,player.position[-1],[0,-20])) in player.position) or player.position[-1][-1] - 20 < 20)) or (
            player.x_change == -20 and player.y_change == 0 and ((list(map(add,player.position[-1],[0,20])) in player.position) or
            player.position[-1][-1] + 20 >= (game.game_height-20))), #danger left


            player.x_change == -20,  # move left
            player.x_change == 20,  # move right
            player.y_change == -20,  # move up
            player.y_change == 20,  # move down
            food.x_food < player.x,  # food left
            food.x_food > player.x,  # food right
            food.y_food < player.y,  # food up
            food.y_food > player.y  # food down
            ]

        for i in range(len(state)):
            if state[i]:
                state[i]=1
            else:
                state[i]=0

        return np.asarray(state)

    def set_reward(self, player, crash):
        self.reward = 0
        if crash:
            self.reward = -10
            return self.reward
        if player.eaten:
            self.reward = 10
        return self.reward

    def network(self, weights=None):
        model = Sequential()
        model.add(Dense(output_dim=120, activation='relu', input_dim=11))
        model.add(Dropout(0.15))
        model.add(Dense(output_dim=120, activation='relu'))
        model.add(Dropout(0.15))
        model.add(Dense(output_dim=120, activation='relu'))
        model.add(Dropout(0.15))
        model.add(Dense(output_dim=3, activation='softmax'))
        opt = Adam(self.learning_rate)
        model.compile(loss='mse', optimizer=opt)

        if weights:
            model.load_weights(weights)
            #print(type(weights))
        return model

    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))

    def replay_new(self, memory):
        if len(memory) > 1000:
            minibatch = random.sample(memory, 1000)
        else:
            minibatch = memory
        for state, action, reward, next_state, done in minibatch:
            target = reward
            if not done:
                target = reward + self.gamma * np.amax(self.model.predict(np.array([next_state]))[0])
            target_f = self.model.predict(np.array([state]))
            target_f[0][np.argmax(action)] = target
            self.model.fit(np.array([state]), target_f, epochs=1, verbose=0)

    def train_short_memory(self, state, action, reward, next_state, done):
        target = reward
        if not done:
            target = reward + self.gamma * np.amax(self.model.predict(next_state.reshape((1, 11)))[0])
        target_f = self.model.predict(state.reshape((1, 11)))
        target_f[0][np.argmax(action)] = target
        self.model.fit(state.reshape((1, 11)), target_f, epochs=1, verbose=0)

Using TensorFlow backend.


In [8]:
list(map(add,[20,0],[20,0]))

[40, 0]

In [124]:
A=[]
A.append([220,80])
x_=0
y_=-20
g_w=440
g_h=440

In [ ]:
(player.x_change == 20 and player.y_change == 0 and ((list(map(add, player.position[-1], [20, 0])) in player.position) or
(player.position[-1][0] + 20) >= (game.game_width - 20) or (player.position[-1][1] == food.y_non_food and (player.position[-1][0] + 20) == food.x_non_food))) or (player.x_change == -20 and player.y_change == 0 and ((list(map(add, player.position[-1], [-20, 0])) in player.position)) or
(player.position[-1][0] - 20) < 20 or (player.position[-1][1] >=80 and player.position[-1][1]<=120 and player.position[-1][0] - 20 <= 240) or (player.position[-1][1] == food.y_non_food and (player.position[-1][0] - 20) == food.x_non_food)) or (player.x_change == 0 and player.y_change == -20 and ((list(map(add, player.position[-1], [0, -20])) in player.position)) or
(player.position[-1][-1] - 20) < 20 or (player.position[-1][0]<=240 and player.position[-1][1] - 20 <= 120 and player.position[-1][1] - 20 >= 80) or (player.position[-1][0] == food.x_non_food and (player.position[-1][1] - 20) == food.y_non_food)) or (player.x_change == 0 and player.y_change == 20 and ((list(map(add, player.position[-1], [0, 20])) in player.position) or
(player.position[-1][-1] + 20) >= (game.game_height-20) or (player.position[-1][0]<=240 and player.position[-1][1] + 20 <= 120 and player.position[-1][1] + 20 >= 80) or (player.position[-1][0] == food.x_non_food and (player.position[-1][1] + 20) == food.y_non_food)))

In [ ]:
(player.x_change == 0 and player.y_change == -20 and ((list(map(add,player.position[-1],[20, 0])) in player.position) or
player.position[ -1][0] + 20 > (game.game_width-20) or (player.position[-1][0] == food.x_non_food and (player.position[-1][1] - 20) == food.y_non_food))) or (player.x_change == 0 and player.y_change == 20 and ((list(map(add,player.position[-1], [-20,0])) in player.position) or
player.position[-1][0] - 20 < 20 or (player.position[-1][1] +20>=80 and player.position[-1][1] +20<=120 and player.position[-1][0]<= 240) or (player.position[-1][0] == food.x_non_food and (player.position[-1][1] + 20) == food.y_non_food))) or (player.x_change == -20 and player.y_change == 0 and ((list(map(add,player.position[-1],[0,-20])) in player.position) or
player.position[-1][-1] - 20 < 20 or (player.position[-1][0] -20<=240 and player.position[-1][1]<= 120 and player.position[-1][1]>= 80) or (player.position[-1][1] == food.y_non_food and (player.position[-1][0] - 20) == food.x_non_food))) or (player.x_change == 20 and player.y_change == 0 and ((list(map(add,player.position[-1],[0,20])) in player.position) or
player.position[-1][-1] + 20 >= (game.game_height-20) or (player.position[-1][0] +20<=240 and player.position[-1][1]<= 120 and player.position[-1][1]>= 80) or (player.position[-1][1] == food.y_non_food and (player.position[-1][0] + 20) == food.x_non_food)))

In [ ]:
(player.x_change == 0 and player.y_change == 20 and ((list(map(add,player.position[-1],[20,0])) in player.position) or player.position[-1][0] + 20 > (game.game_width-20))) or (player.x_change == 0 and player.y_change == -20 and
((list(map(add, player.position[-1],[-20,0])) in player.position) or player.position[-1][0] - 20 < 20 or (player.position[-1][0] >=240 and player.position[-1][1] - 20 <=120 and player.position[-1][0] - 20 >= 80))) or (player.x_change == 20
and player.y_change == 0 and ((list(map(add,player.position[-1],[0,-20])) in player.position) or player.position[-1][-1] - 20 < 20 or (player.position[-1][0] >=0 and player.position[-1][0]<=240 and player.position[-1][1] - 20 <= 120 and
player.position[-1][1] - 20 >= 80))) or (player.x_change == -20 and player.y_change == 0 and ((list(map(add,player.position[-1],[0,20])) in player.position) or player.position[-1][-1] + 20 >= (game.game_height-20) or (player.position[-1][0] >=0 and
player.position[-1][0]<=240 and player.position[-1][1] + 20 <= 120 and player.position[-1][1] + 20 >= 80)))

In [ ]:
(player.x_change == 0 and player.y_change == 20 and ((list(map(add,player.position[-1],[20,0])) in player.position) or player.position[-1][0] + 20 > (game.game_width-20) or (player.position[-1][1] == food.y_non_food and (player.position[-1][0] + 20) == food.x_non_food))) or (player.x_change == 0 and player.y_change == -20 and
((list(map(add, player.position[-1],[-20,0])) in player.position) or player.position[-1][0] - 20 < 20 or (player.position[-1][0] - 20 >=240 and player.position[-1][1]<=120 and player.position[-1][1]>= 80) or (player.position[-1][1] == food.y_non_food and (player.position[-1][0] - 20) == food.x_non_food))) or (player.x_change == 20
and player.y_change == 0 and ((list(map(add,player.position[-1],[0,-20])) in player.position) or player.position[-1][-1] - 20 < 20 or (player.position[-1][0]<=240 and player.position[-1][1] - 20 <= 120 and
player.position[-1][1] - 20 >= 80) or (player.position[-1][0] == food.x_non_food and (player.position[-1][1] - 20) == food.y_non_food))) or (player.x_change == -20 and player.y_change == 0 and ((list(map(add,player.position[-1],[0,20])) in player.position) or player.position[-1][-1] + 20 >= (game.game_height-20) or (player.position[-1][0]<=240 and player.position[-1][1] + 20 <= 120 and player.position[-1][1] + 20 >= 80) or (player.position[-1][0] == food.x_non_food and (player.position[-1][1] + 20) == food.y_non_food)))

In [ ]:
 or (player.position[-1][1] == food.y_non_food and (player.position[-1][0] + 20) == food.x_non_food)